In [1]:
#!pip3 install nltk
#!pip3 install textblob


#!pip3 install spaCy
#!pip3 install gensim
#!pip3 install pattern

#!pip3 install speechrecognition

#!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

#!pip3 install speechrecognition
#!brew install portaudio
#!pip3 install pyaudio

In [2]:
from pymongo import MongoClient
import pandas as pd
import requests

In [56]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy
from spacy import displacy

import tools.sentiments as sent

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
import en_core_web_sm

/usr/local/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.4). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
import speech_recognition as sr 
import pyaudio

In [6]:
client = MongoClient("mongodb://localhost/Simpsons") 
db = client.get_database()
messages_coll = db["messages"]
char_coll = db["characters"]
group_coll = db["chats"]

In [11]:
simpsonsdf = pd.DataFrame(homer_messages, columns=["Messages"])
simpsonsdf.head()

,Messages
0,Never thrown a party? What about that big bash...
1,"Bart didn't get one vote?! Oh, this is the wor..."
2,Oh.
3,And?
4,"Hey, just because I don't care doesn't mean I ..."


In [12]:
todo = pd.DataFrame(list(messages_coll.find()))
todo.head()

,_id,chat_name,participant_name,message
0,5fc4ee229900a48ad2ee83b0,elementary_school,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,5fc4ee229900a48ad2ee83b1,elementary_school,Miss Hoover,I don't know. Although I'd sure like to talk t...
2,5fc4ee229900a48ad2ee83b2,elementary_school,Miss Hoover,"Thank you, Ralph. Very graphic. Lisa Simpson? ..."
3,5fc4ee229900a48ad2ee83b3,elementary_school,Miss Hoover,Dear God!
4,5fc4ee229900a48ad2ee83b4,elementary_school,Miss Hoover,I question the educational value of this assem...


In [13]:
todo = todo[['participant_name', 'chat_name','message']]

In [14]:
todo.groupby('participant_name')
todo.head()

,participant_name,chat_name,message
0,Miss Hoover,elementary_school,"No, actually, it was a little of both. Sometim..."
1,Miss Hoover,elementary_school,I don't know. Although I'd sure like to talk t...
2,Miss Hoover,elementary_school,"Thank you, Ralph. Very graphic. Lisa Simpson? ..."
3,Miss Hoover,elementary_school,Dear God!
4,Miss Hoover,elementary_school,I question the educational value of this assem...


NLTK

### I write here the function so you can check how it works.

```
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    pol = polarity['compound']
    return pol
```


### We incorporate a column to our dataframe so we apply the sentimentAnalysis function which let us obtain the polarity  scores compound of each message.

In [17]:
todo['sentiment_compound'] = todo['message'].apply(sent.sentimentAnalysis)
todo.head()

,participant_name,chat_name,message,sentiment_compound
0,Miss Hoover,elementary_school,"No, actually, it was a little of both. Sometim...",0.0772
1,Miss Hoover,elementary_school,I don't know. Although I'd sure like to talk t...,0.5859
2,Miss Hoover,elementary_school,"Thank you, Ralph. Very graphic. Lisa Simpson? ...",0.6553
3,Miss Hoover,elementary_school,Dear God!,0.6114
4,Miss Hoover,elementary_school,I question the educational value of this assem...,0.3400


### We can see the mean of the sentiment_compound for each character.

In [18]:
todo.groupby(['participant_name'])['sentiment_compound'].mean()

participant_name
Barney Gumble           0.091576
Bart Simpson            0.052006
C. Montgomery Burns     0.096846
Carl Carlson            0.089219
Chief Wiggum            0.059382
Dr. Julius Hibbert      0.054617
Fat Tony                0.088740
Goodman Skinner        -0.000675
Homer Simpson           0.075905
Jimbo Jones             0.059743
Lenny Leonard           0.086263
Lisa Simpson            0.062032
MRS KRABAPPEL           0.000000
Maggie Simpson         -0.001166
Marge Simpson           0.082378
Martin Prince           0.107417
Mayor Joe Quimby        0.054295
Milhouse Van Houten     0.078111
Miss Hoover             0.144571
Moe Szyslak             0.092434
Ned Flanders            0.115441
Nelson Muntz            0.026378
Ralph Wiggum            0.058228
Rev. Timothy Lovejoy    0.113937
Waylon Smithers         0.062927
Name: sentiment_compound, dtype: float64

### We obtain the messages list

In [19]:
mess_lst = todo['message'].tolist()

In [20]:
#mess_lst

### We obtain the sentiment parameters for each message of the messages list.

In [21]:
sentiments = {}
for mess in mess_lst:
    blob = TextBlob(mess)
    sentiments[mess] = blob.sentiment


In [54]:
#sentiments

### I write here the function so you can check how it works.

```
def sentimentsAndPolarity(message):
    polarity_and_sentiment_mess = []
    for mess in mess_lst:
        blob = TextBlob(mess)
        polarity_and_sentiment_mess.append({'message': mess,
                                           'sentiment': {'polarity': blob.sentiment.polarity, 
                                                        'subjectivity': blob.sentiment.subjectivity}, 
                                           'polarity':sia.polarity_scores(mess)})
    return polarity_and_sentiment_mess
```

In [40]:
nlp_list = sent.sentimentsAndPolarity(mess_lst)

In [41]:
print(nlp_list[0])

{'message': "No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.", 'sentiment': {'polarity': -0.021875, 'subjectivity': 0.5}, 'polarity': {'neg': 0.069, 'neu': 0.852, 'pos': 0.079, 'compound': 0.0772}}


### Now we introduce the sentiment and polarity fields for each message of the messages collection

#### I need to say that because of the huge amount of time it took to create those fileds for the thousands of messages, after more than four hours I obtain the fields for all the messages except for ninety thousand of them (more or less), so, when using the endpoint, there are some information you may not find. (Sorry 🥺)

In [52]:
url_nlp = "http://localhost:5000/new/message/nlp"
for nlp in nlp_list:
    requests.post(url_nlp, json=nlp)